# Build custom SageMaker image for geospatial processing

The [SageMaker Distribution](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-distribution.html) is a pre-built container for Studio JupyterLab apps that includes:
- Deep learning frameworks (PyTorch, TensorFlow, Keras) for a GPU or CPU distribution type
- Python ML packages (numpy, scikit-learn, pandas)
- JupyterLab IDE
  
All components are compatible and up-to-date. The SageMaker Distribution enables ML practitioners to get started quickly with their ML development in SageMaker AI Studio. 

If you need functionality that is different than what's provided by SageMaker Distribution, you can bring your own image with your custom extensions and packages. The custom images based on the SageMaker Distribution can work both as interactive notebooks in SageMaker AI Studio and for SageMaker AI jobs like processing or training jobs, enabling smooth transitions from development in a notebook to production.

This notebook demonstrate how to create a custom SageMaker image for Studio notebooks with specific geospatial processing libraries.

See the  [Create custom images for geospatial analysis with Amazon SageMaker Distribution in Amazon SageMaker Studio](https://aws.amazon.com/blogs/machine-learning/create-custom-images-for-geospatial-analysis-with-amazon-sagemaker-distribution-in-amazon-sagemaker-studio/) blog post for detailed overview of a custom image creation requirements and workflow.

## Setup environment

In [83]:
import os
import json
import boto3
import sagemaker
from packaging import version

In [84]:
sm_client = boto3.client('sagemaker')

In [85]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

SageMaker domain id: d-cvmfjrqvekyk
Space name: geo-2
User profile: studio-user-geo-270fdb90
Stored 'domain_id' (str)
Stored 'space_name' (str)
Stored 'user_profile_name' (str)


In [86]:
role = sagemaker.get_execution_role()

### Check if docker access is enabled in the domain

In [87]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

The docker access is ENABLED in the domain d-cvmfjrqvekyk


<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. See the following instructions how to do it.
</div>

In [97]:
print(f"Domain id: {domain_id}")

Domain id: d-cvmfjrqvekyk


### Enable docker access for the SageMaker domain

<div class="alert alert-info">You only need this section if the docker access is not enabled in the domain.
</div>

To update domain settings, you can use **one** of the following options.

#### Option 1: run `update_domain` in the notebook
You need `sagemaker:UpdateDomain` permission in the execution role to run the following code in the notebook.
If you have the corresponding permissions, run the following code in a notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
        }
    }
)
```

#### Option 2: run `aws sagemaker` CLI in the  terminal
Make sure you run `AWS CLI` in the terminal where you have the corresponding permission `sagemaker:UpdateDomain`. 
Run the following command:

```
aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update DockerSettings={EnableDockerAccess='ENABLED'}
```

For example, you can run the command above in the [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) in your AWS account.

### Install Docker

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version

## Get the current SageMaker Image

The custom image is based on the current SageMaker image. See the [Amazon SageMaker Distribution GitHub](https://github.com/aws/sagemaker-distribution/) for the full list of SageMaker distribution images.

In [89]:
# Retrieve the SageMaker image for the current space, extract the type (cpu/gpu) and the version
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']
sm_image = resource_spec['SageMakerImageArn']
sm_dist_type = sm_image.split('-')[-1]
sm_image_version = version.parse(resource_spec['SageMakerImageVersionAlias'])
sm_image_version = f'{sm_image_version.major}.{sm_image_version.minor}'

print(f"""
SageMaker image: \033[1m{sm_image}\033[0m
SageMaker image type: \033[1m{sm_dist_type}\033[0m
SageMaker image version: \033[1m{sm_image_version}\033[0m
""")


SageMaker image: arn:aws:sagemaker:us-west-2:542918446943:image/sagemaker-distribution-cpu
SageMaker image type: cpu
SageMaker image version: 1.11



In [90]:
repo_name = "smd-custom-geo"
image_name = f'{sm_image_version}-{sm_dist_type}'

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = metadata["ResourceArn"].split(":")[3]
ecr_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{repo_name}:{image_name}"

print(f'Custom image ECR URI: {ecr_uri}')

Custom image ECR URI: 070672918432.dkr.ecr.us-west-2.amazonaws.com/smd-custom-geo:1.11-cpu


## Build a custom Docker image

There are several approaches to build a custom SageMaker image. For any approach you need to implement the following steps:
1. Create a Dockerfile extending from SageMaker Distribution and configured with required packages, dependencies, and environments
2. Configure an Amazon ECR repository to host the images
3. Build and push the image to ECR repository
4. Attach the image to SageMaker AI Studio domain

For simplicity, this notebook builds and pushes the image using bash script in code cells. For a real-world scenario you need to implement an automated image building and testing pipeline, and use Infrastructure as Code to automate the deployment.

For more details refer to this [example](https://github.com/aws-samples/sagemaker-custom-image-for-geospatial-analytics) and [SageMaker AI documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-jl-admin-guide-custom-images.html).

In [91]:
# %%bash

# docker system prune -af 

In [92]:
# Pass the variables to the build script
os.environ['REPO_NAME'] = repo_name
os.environ['SMD_DIST_TYPE'] = sm_dist_type
os.environ['SMD_VERSION'] = sm_image_version

In [93]:
%%bash

set -e

# Region, defaults to us-east-1
REGION=$AWS_DEFAULT_REGION
REGION=${REGION:-us-east-1}

ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)

echo "REPO_NAME: $REPO_NAME"
echo "ACCOUNT_ID: $ACCOUNT_ID"
echo "REGION: $REGION"
echo "SMD_DIST_TYPE: $SMD_DIST_TYPE"
echo "SMD_VERSION: $SMD_VERSION"

TAG=${REPO_NAME}:${SMD_VERSION}-${SMD_DIST_TYPE}
ECR_URI="${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/${TAG}"

echo "IMAGE TAG: $TAG"
echo "ECR TARGET: ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/${TAG}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${REPO_NAME}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${REPO_NAME}" > /dev/null
fi

# Login to ECR
aws ecr get-login-password --region ${REGION} | docker login --username AWS --password-stdin ${ECR_URI}

# Build and push the image
set -x
docker build --network sagemaker -f Dockerfile --build-arg SMD_DISTRIBUTION_TYPE=$SMD_DIST_TYPE --build-arg SMD_VERSION=$SMD_VERSION -t $TAG .
set +x

docker tag $TAG ${ECR_URI}
docker push ${ECR_URI}

echo ""
echo "Created image pushed to ECR image URI: $ECR_URI"
echo ""
echo "Done"

REPO_NAME: smd-custom-geo
ACCOUNT_ID: 070672918432
REGION: us-west-2
SMD_DIST_TYPE: cpu
SMD_VERSION: 1.11
IMAGE TAG: smd-custom-geo:1.11-cpu
ECR TARGET: 070672918432.dkr.ecr.us-west-2.amazonaws.com/smd-custom-geo:1.11-cpu


WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


+ docker build --network sagemaker -f Dockerfile --build-arg SMD_DISTRIBUTION_TYPE=cpu --build-arg SMD_VERSION=1.11 -t smd-custom-geo:1.11-cpu .


Sending build context to Docker daemon  466.9kB
Step 1/16 : ARG SMD_DISTRIBUTION_TYPE=cpu
Step 2/16 : ARG SMD_VERSION=1.11
Step 3/16 : FROM public.ecr.aws/sagemaker/sagemaker-distribution:$SMD_VERSION-$SMD_DISTRIBUTION_TYPE
 ---> 065bb6c91b29
Step 4/16 : ARG NB_USER="sagemaker-user"
 ---> Using cache
 ---> 08935b23c53f
Step 5/16 : ARG NB_UID=1000
 ---> Using cache
 ---> 3c440d003582
Step 6/16 : ARG NB_GID=100
 ---> Using cache
 ---> c9936fcbc137
Step 7/16 : ENV MAMBA_USER=$NB_USER
 ---> Using cache
 ---> 84eedd489564
Step 8/16 : USER $ROOT
 ---> Running in 684d79021085
 ---> Removed intermediate container 684d79021085
 ---> c28fc13a176c
Step 9/16 : ARG CPLUS_INCLUDE_PATH=/usr/include/gdal
 ---> Running in 5ea74bfa5342
 ---> Removed intermediate container 5ea74bfa5342
 ---> 1db8c219163e
Step 10/16 : ARG C_INCLUDE_PATH=/usr/include/gdal
 ---> Running in 37c297b6dcad
 ---> Removed intermediate container 37c297b6dcad
 ---> 58950e79f31c
Step 11/16 : RUN apt-get --allow-releaseinfo-change up

+ set +x


2cb1624d7a19: Preparing
354e6f05d233: Preparing
6a88591a8a36: Preparing
110d782df9ad: Preparing
d1ae633ae001: Preparing
203ec793781a: Preparing
be6d5ddf49bc: Preparing
e49b98b9e44e: Preparing
1d6d8d1bb97a: Preparing
39a44b600aad: Preparing
98dbf19a3b78: Preparing
26d1ebfbe2f3: Preparing
55892c4afa77: Preparing
4d42a6f47a15: Preparing
4abb79bee12f: Preparing
fc3f448661e8: Preparing
6e4c58e850ea: Preparing
c7f3bd79b36d: Preparing
83f735c752b8: Preparing
442d3d55c177: Preparing
2d7ba4e6f66a: Preparing
3989a2820343: Preparing
3d1ffdd76676: Preparing
5988af297b69: Preparing
79cb5ae2c0d9: Preparing
33c2c3f9b811: Preparing
203ec793781a: Waiting
a5bc85445705: Preparing
be6d5ddf49bc: Waiting
e49b98b9e44e: Waiting
492abaa0bb13: Preparing
4d42a6f47a15: Waiting
1d6d8d1bb97a: Waiting
7964a87a2ca4: Preparing
4abb79bee12f: Waiting
39a44b600aad: Waiting
bc5572752fdf: Preparing
98dbf19a3b78: Waiting
fc3f448661e8: Waiting
26d1ebfbe2f3: Waiting
442d3d55c177: Waiting
fe2b0873d113: Preparing
6e4c58e850ea: 

After a successful build and push, list images in the ECR. You must see the latest built image.

In [94]:
!aws ecr list-images --repository-name {repo_name}

{
    "imageIds": [
        {
            "imageDigest": "sha256:01f97885b02f94e1ac6690d238df47f059a94fcbca2b6d9c9d19cc0204ab2a88"
        },
        {
            "imageDigest": "sha256:af802752dc544cffda9eb73f69a93d81d173ec1a62c6d1ead260b0e369254830",
            "imageTag": "latest-cpu"
        },
        {
            "imageDigest": "sha256:6d82f57bf172e8a27e416cbdad6eb5c6bbba1e6a45f07dcf24df10b85fcd4e01",
            "imageTag": "1.11-cpu"
        }
    ]
}


## Attach image to the SageMaker domain

Having the image in the ECR repository in your account, you can now attach it to the SageMaker AI domain.

In [95]:
# Output image URI
ecr_uri

'070672918432.dkr.ecr.us-west-2.amazonaws.com/smd-custom-geo:1.11-cpu'

Follow the instructions:

1. Open the [SageMaker AI console](https://console.aws.amazon.com/sagemaker)
2. Select **Admin configurations** on the left pane, choose **Domains**
3. Select the domain to which you want to attach the image
4. Select the **Environment** tab
5. In the section **Custom images for personal Studio apps**, select **Attach image**
6. Select **New image** and enter the image URI to the **Enter an ECR image URI**, select **Next**
7. Enter an **Image name** and **Image display name**, for example `smd-geo` and `SageMaker Distribution Geo 1.11 CPU`
8. Select **JupyterLab image** as application type and click on **Submit**

## Create a JupyterLab app with the custom image

Follow the instructions:
1. Open Studio
2. Select **JupyterLab**
3. Select **+ Create JupyterLab space** and enter a name for your new space, for example `geo-custom`, select **Create space**
4. Select the custom image you attached to SageMaker Studio as **Image**
5. Select **Run Space**

![](../img/select-custom-image.png)

Wait until the space change status to `Running` and select **Open**. This will open a JupyterLab app in a new browser tab.

## Use the custom image

Now you can use the custom image in one of two ways:
1. Use the image to run JupyterLab notebook kernels
2. Use the image to run SageMaker training and processing jobs

For examples change to the [`06_smd_custom_geospatial`](../06_smd_custom_geospatial.ipynb) notebook.